In [1]:
# Train YOLO model.
# Images and labels are expected in '../data/512/'.

# Convert json label files to expected format.
# Create training / validation sets.

In [ ]:
import ultralytics
from ultralytics import YOLO
ultralytics.checks()
import glob
from sklearn.model_selection import train_test_split
import json
import os
import shutil

In [3]:
DATASET_PATH="../data/512"
YOLO11N_MODEL_PATH = "../models/yolo11n-obb.pt"

In [ ]:
# After labelling, create datasets
files = glob.glob(DATASET_PATH+"/*.jpg")
print(DATASET_PATH+"/*.jpg")

files_train, files_validation = train_test_split(files, test_size=0.2, random_state=1)
print("Train Set:", len(files_train))
print("Validation Set:", len(files_validation))

In [5]:
# Convert Anylabeling label file to expected format for YOLO11
def makeYoloConfig(fname):
    cfg = json.load(open(fname))
    lines = []
    imageHeight = cfg['imageHeight']
    imageWidth = cfg['imageWidth']

    # class x_center y_center width height
    # 0 0.5 0.2 0.1 0.1 0.2 0.3 0.3 0.2 0.1  
    # Each line in the text file represents one object with its class label, followed by the polygon coordinates.
    # Each polygon point is normalized x and y coordinates (between 0 and 1) relative to the image dimensions.
    for label_idx, label in enumerate(cfg['shapes']):
        line = ['0'] # NOTE: Assumes we only have one label. Index: 0 refers to "streak" in streaks-512.yaml
        for point in label['points']:
            line.append(point[0]/imageWidth)
            line.append(point[1]/imageHeight)
        lines.append(" ".join(str(num) for num in line))
    
    f_out = open(fname.replace(".json", ".txt"), "w")
    for line in lines:
        f_out.write(line + "\n")
    f_out.close()

In [6]:
# Move files
IMAGES_TRAIN_PATH = DATASET_PATH + "/train"
IMAGES_Validation_PATH = DATASET_PATH + "/validation"

shutil.rmtree(IMAGES_TRAIN_PATH, ignore_errors=True)
shutil.rmtree(IMAGES_Validation_PATH, ignore_errors=True)

os.makedirs(IMAGES_TRAIN_PATH + "/images/", exist_ok=True)
os.makedirs(IMAGES_TRAIN_PATH + "/labels/", exist_ok=True)
os.makedirs(IMAGES_Validation_PATH + "/images/", exist_ok=True)
os.makedirs(IMAGES_Validation_PATH + "/labels/", exist_ok=True)

for file in files_train:
    try:
        file_json = file.replace("jpg", "json")
        file_txt = file.replace("jpg", "txt")
        makeYoloConfig(file_json)
        shutil.copy(file, IMAGES_TRAIN_PATH + "/images/")
        shutil.copy(file_txt, IMAGES_TRAIN_PATH + "/labels/")

    except Exception as e:
        print(file_json)
        print(e)
        

for file in files_validation:
    try:
        file_json = file.replace("jpg", "json")
        file_txt = file.replace("jpg", "txt")
        makeYoloConfig(file_json)
        shutil.copy(file, IMAGES_Validation_PATH + "/images/")
        shutil.copy(file_txt, IMAGES_Validation_PATH + "/labels/")
    except Exception as e:
        print(file_json)
        print(e)

# TODO: If we had any errors here, exclude the images so they don't count as backgrounds

In [ ]:
model = YOLO(YOLO11N_MODEL_PATH)
results = model.train(data="streaks-512.yaml", epochs=200, batch=16, augment=True, imgsz=512)

In [ ]:
shutil.copy("{}/{}".format(results.save_dir, "/weights/best.pt"), "../models/streaks.pt")